Use this utlity to update the returns and std_dev fields within investment-options.csv

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

In [4]:
# Set size of inline plots.
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [5]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = False

In [6]:
# Read in investment options input csv.
df = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])
df

,Investment Option,Description,Asset Class,1 mo,3 mo,1 Yr,3 Yr,5 Yr,Vola,DS Vola,Std Dev
0,EHC,Encompass Health,US Stocks:Healthcare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RF,Regions Financial,US Stocks:Financials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VMC,Vulcan Materials Company,US Stocks:Materials,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HIBB,Hibbett Sports,US Stocks:Consumer Discretionary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADTN,ADTRAN,US Stocks:Technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [8]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

.....


In [10]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,ADTN,HIBB,RF,VMC,EHC
Date,,,,,
2015-01-02,18.44,46.80,8.51,62.36,32.32
2015-01-05,18.40,45.51,8.22,60.64,32.21
2015-01-06,18.13,44.87,7.94,60.50,31.96
2015-01-07,18.22,47.18,7.99,61.65,32.94
2015-01-08,18.15,48.08,8.11,63.57,33.72
...,...,...,...,...,...
2022-04-18,17.92,44.57,20.92,173.99,71.41
2022-04-19,18.07,47.45,21.68,177.21,71.92
2022-04-20,18.35,47.28,21.97,182.63,72.94


In [12]:
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

ADTN     3.69
HIBB   -38.01
RF       5.98
VMC     -1.35
EHC    -23.60
dtype: float64

In [10]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [11]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [12]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [13]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()

In [14]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change()

In [15]:
# Calculate 3 year annualized standard deviation.
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)

In [16]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')